In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# Value Based Search
## Creating the client
Setting the timezone is important as search results will be given in this timezone. If we want to save searches, we should also add user authentication (username and password).

In [2]:
import keyring
from trendminer_interface import TrendMinerClient

url = "https://cs.trendminer.net"
client_id = "tmInterfaceClient"
username = "wdaniels"

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username),
    tz="Europe/Brussels"
)

## Creating a search
A number of input formats and parameters are accepted.

In [3]:
# the most basic
vbs = client.search.value("[CS]BA:LEVEL.1 > 0")

# more explicit
level = client.tag.from_name("[CS]BA:LEVEL.1")
vbs = client.search.value((level, ">", 0))

# multiple queries, additional arguments
vbs = client.search.value(
    queries=[
        (level, ">", 0),
        "[CS]BA:ACTIVE.1 = Active",  # we can mix different query formats
        "[CS]BA:PHASE.1 constant",
    ],
    operator="AND",  # the default, can also be set to "OR"
    duration="15m",  # minimal duration, default is 2x index resolution
)

vbs

<< ValueBasedSearch | New Search >>

## Executing the search
Search period is given at execution of the search. The results are given as a list of intervals.

In [4]:
results = vbs.get_results(("2022-12-01", "2023-01-01"))  # results for December 2023

results = vbs.get_results("8h") # results for the last 8 hours

results

[<< Interval | 2023-01-12 05:22:00+01:00 | 1:10:00 >>,
 << Interval | 2023-01-12 06:43:00+01:00 | 1:09:00 >>,
 << Interval | 2023-01-12 08:04:00+01:00 | 1:08:00 >>,
 << Interval | 2023-01-12 09:25:00+01:00 | 1:10:00 >>,
 << Interval | 2023-01-12 10:46:00+01:00 | 1:09:00 >>,
 << Interval | 2023-01-12 12:07:00+01:00 | 1:01:00 >>]

### Filters
We can give a list of intervals to exclude from the search (i.e., intervals to filter out). These can be defined manually, but can also be the results of a previous search.

In [5]:
search_interval = client.interval("2023-01-01 10:00", "2023-01-01 22:00")

excluded = [
    client.interval("2023-01-01 12:00", "2023-01-01 14:00"),
    client.interval("2023-01-01 18:00", "2023-01-01 20:00"),
]

vbs.get_results(search_interval, excluded_intervals=excluded)   

[<< Interval | 2023-01-01 10:40:00+01:00 | 0:21:00 >>,
 << Interval | 2023-01-01 14:43:00+01:00 | 0:21:00 >>,
 << Interval | 2023-01-01 16:04:00+01:00 | 0:19:00 >>,
 << Interval | 2023-01-01 17:26:00+01:00 | 0:18:00 >>,
 << Interval | 2023-01-01 20:07:00+01:00 | 0:20:00 >>,
 << Interval | 2023-01-01 21:28:00+01:00 | 0:18:00 >>]

If wanted to search only in a given list of intervals (i.e., 'filter in'), we could have inverted the intervals first

In [6]:
included = [
    client.interval("2023-01-01 12:00", "2023-01-01 14:00"),
    client.interval("2023-01-01 18:00", "2023-01-01 20:00"),
]

excluded = client.interval.invert(included, span=search_interval)
excluded

[<< Interval | 2023-01-01 10:00:00+01:00 | 2:00:00 >>,
 << Interval | 2023-01-01 14:00:00+01:00 | 4:00:00 >>,
 << Interval | 2023-01-01 20:00:00+01:00 | 2:00:00 >>]

## Adding calculations
Calculations can be added in the search definition. Calculations need to be defined as a dict with `{key: (tag, operation), ...}` structure.

In [7]:
calculations = {
            "test": ("[CS]BA:LEVEL.1", "max"),
            "test2": ("[CS]BA:ACTIVE.1", "start"),
        }

vbs = client.search.value(
    queries=["[CS]BA:LEVEL.1 > 10"],
    calculations=calculations,
)

Calculations results are added to the returned intervals, under the given keys

In [8]:
results = vbs.get_results("1h")
results

[<< Interval | 2023-01-12 12:12:00+01:00 | 0:56:00  | test: 41.543026, test2: Active >>]

In [9]:
results[0]["test2"]

'Active'

## Saving and loading searches
If we want to save a search, we should add at least a name, but potentially also a description and folder when defining the search.

In [10]:
# HIDDEN - create folder
folder = client.folder.from_path("ExistingFolder", create_new=True)

In [11]:
vbs = client.search.value(
    queries=[("[CS]BA:Active.1", "=", "Active")],
    duration="1h",
    name="Active Batch",
    description="full length active batches",
    folder="ExistingFolder",
    calculations=calculations,
)

vbs

<< ValueBasedSearch | Active Batch >>

 We can save to the appliance via a post request

In [12]:
vbs.post()

The easiest way to load a search is via its path in the work organizer

In [13]:
loaded_vbs = client.search.value.from_path("ExistingFolder/Active Batch")
loaded_vbs.get_results("4h")

[<< Interval | 2023-01-12 09:25:00+01:00 | 1:10:00  | test: 43.155704, test2: Active >>,
 << Interval | 2023-01-12 10:46:00+01:00 | 1:10:00  | test: 39.11838, test2: Active >>,
 << Interval | 2023-01-12 12:07:00+01:00 | 1:04:00  | test: 41.543026, test2: Active >>]

A search can always be updated via a put request

In [14]:
loaded_vbs.name = "updated name"
loaded_vbs.put()

And removed from the appliance with a delete request

In [15]:
loaded_vbs.delete()

In [16]:
# HIDDEN - create folder
folder.delete()